In [17]:
import numpy as np
import pandas as pd


In [19]:
books = pd.read_csv(
    "../data/dataset_books.csv",
    encoding="latin-1",
    on_bad_lines="skip",
    # sep=";",
    # delimiter=";"
    # error_bad_lines=False,
)


books.shape


books.head(2)


/tmp/ipykernel_483638/1615894012.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...


In [ ]:
books.columns

books.columns = books.columns.str.strip()


books = books[
    [
        "ISBN",
        "Book-Title",
        "Book-Author",
        "Year-Of-Publication",
        "Publisher",
    ]
]


books.rename(
    columns={
        "ISBN": "ISBN",
        "Book-Title": "title",
        "Book-Author": "author",
        "Year-Of-Publication": "yop",
        "Publisher": "publisher",
    },
    inplace=True,
)

books.head()


In [ ]:
print(books.columns.tolist())
for col in books.columns:
    print(repr(col))


In [20]:
users = pd.read_csv(
    "../data/dataset_users.csv",
    on_bad_lines="skip",
    encoding="latin-1",
)

users.head(2)
# users.columns


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [ ]:
users.rename(
    columns={
        "User-ID": "user_id",
        "Location": "location",
        "Age": "age",
    },
    inplace=True,
)

users.head(2)


In [ ]:
ratings = pd.read_csv(
    "../data/dataset_ratings.csv",
    on_bad_lines="skip",
    encoding="latin-1",
)


ratings.head(2)


In [ ]:
ratings.columns


ratings.rename(
    columns={
        "User-ID": "user_id",
        "ISBN": "ISBN",
        "Book-Rating": "rating",
    },
    inplace=True,
)


In [ ]:
print(books.shape)
print(users.shape)
print(ratings.shape)


In [ ]:
ratings.head()

uid_high_ratings = ratings["user_id"].value_counts() > 200
# ratings[ratings["user_id"].value_counts() > 200]

# ratings[ratings["user_id"]== 11676].head()


In [ ]:
y = uid_high_ratings[uid_high_ratings].index
y


In [ ]:
ratings = ratings[ratings["user_id"].isin(y)]
ratings


In [ ]:
# ratings.head()

ratings_with_books = ratings.merge(books, on="ISBN")
ratings_with_books.shape

ratings_with_books.groupby("title")


In [ ]:
number_of_ratings = (
    ratings_with_books.groupby("title")["rating"]
    .count()
    .reset_index()
    # .rename(
    #     columns={"ratings": "number_of_ratings"},
    #     # inplace=True,
    # )
)


In [ ]:
number_of_ratings.rename(
    columns={"ratings": "number_of_ratings"},
    inplace=True,
)

print(number_of_ratings)


In [ ]:
final_rating = ratings_with_books.merge(number_of_ratings, on="title")
final_rating.head()


In [ ]:
final_rating = final_rating[final_rating["rating_y"] >= 50]

final_rating.head()


In [ ]:
final_rating.drop_duplicates(["user_id", "title"], inplace=True)

book_pivot = final_rating.pivot_table(
    columns="user_id",
    index="title",
    values="rating_y",
)


book_pivot.head()


NameError: name 'final_rating' is not defined

In [ ]:
# import seaborn as sns
#
#
# sns.scatterplot(book_pivot)


book_pivot.fillna(0, inplace=True)
book_pivot.head()


In [ ]:
from scipy.sparse import csr_matrix

book_sparse = csr_matrix(book_pivot)


book_pivot.to_csv("../models/book_pivot.csv")
type(book_sparse)


NameError: name 'book_pivot' is not defined

In [ ]:
from sklearn.neighbors import NearestNeighbors
import joblib

model = NearestNeighbors(algorithm="brute")
model.fit(book_sparse)


np.where(book_pivot.index == 237)

# save model
joblib.dump(model, "knn_model.pkl")


NameError: name 'book_sparse' is not defined

In [ ]:
book_index = 237

distances, suggestions = model.kneighbors(
    book_pivot.iloc[book_index, :].values.reshape(1, -1)
)


# print(book_pivot)
print(suggestions)
print(distances)


In [ ]:
book_pivot.index[book_index]  # suggestions


In [ ]:
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])


In [ ]:
np.where(book_pivot.index == "Animal Farm")[0][0]


In [ ]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == book_name)[0][0]

    distances, suggestions = model.kneighbors(
        book_pivot.iloc[book_id, :].values.reshape(1, -1)
    )

    # for i in range(len(suggestions)):
    #     if i != 0:
    #         print(f"The suggestions for {book_name}({book_id}) are :\n")
    #         print(book_pivot.index[suggestions[i]])
    #     else:
    #         print("None Found")

    print(f"The suggestions for {book_name}({book_id}) are :\n")

    for idx in suggestions[0][1:]:  # skip index 0 (the book itself)
        print(book_pivot.index[idx])


# recommend_book("Animal Farm")


In [ ]:
recommend_book("Harry Potter and the Chamber of Secrets (Book 2)")
